In [ ]:
import copy
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams["backend"] = 'agg'

In [ ]:
RESULTS_DIR = 'voronoi_results/'
MIQP_NAME = lambda iter_nr: f'binapprox_miqp_iter_{iter_nr}.json'
NLP_NAME = lambda iter_nr: f'nlpsolver_bin_miqp_iter_{iter_nr}.json'

In [ ]:
with open(RESULTS_DIR + 'nlpsolver_rel.json', 'rb') as f:
        nlpsolver_rel = json.load(f)

In [ ]:
iter_nr = 1

In [ ]:
with open(RESULTS_DIR + MIQP_NAME(iter_nr=iter_nr), "rb") as f:
        nlpsolver_miqp = json.load(f)

with open(RESULTS_DIR + NLP_NAME(iter_nr=iter_nr), "rb") as f:
        nlpsolver_nlp_bin = json.load(f)

In [ ]:
def generate_data_df(nlpsolver_json) -> pd.DataFrame:

    u_df = pd.DataFrame(columns=['time_grid']+[f'u_{i}' for i in range(len(nlpsolver_json['data']['u_data'][0]))])
    u_df['time_grid'] = nlpsolver_json['time_points']
    u_df.set_index('time_grid', inplace=True)
    u_df.index = pd.to_datetime(u_df.index).tz_convert('CET')
    tmp = copy.copy(nlpsolver_json['data']['u_data'])
    tmp.append((np.nan * np.zeros(len(nlpsolver_json['data']['u_data'][0]))).tolist())
    u_df[u_df.columns] = tmp
    u_df.columns = ["v_ppsc", "p_mpsc", "v_pssc", "P_g", "mdot_o_hts_b", "mdot_i_hts_b"]

    b_df = pd.DataFrame(columns=['time_grid']+[f'b_{i}' for i in range(len(nlpsolver_json['data']['b_data'][0]))])
    b_df['time_grid'] = nlpsolver_json['time_points']
    b_df.set_index('time_grid', inplace=True)
    b_df.index = pd.to_datetime(b_df.index).tz_convert('CET')
    tmp = copy.copy(nlpsolver_json['data']['b_data'])
    tmp.append((np.nan * np.zeros(len(nlpsolver_json['data']['b_data'][0]))).tolist())
    b_df[b_df.columns] = tmp
    b_df.columns = ["b_ac", "b_fc", "b_hp"]

    x_df = pd.DataFrame(columns=['time_grid']+[f'x_{i}' for i in range(len(nlpsolver_json['data']['x_data'][0]))])
    x_df['time_grid'] = nlpsolver_json['time_points']
    x_df.set_index('time_grid', inplace=True)
    x_df.index = pd.to_datetime(x_df.index).tz_convert('CET')
    x_df[x_df.columns] = nlpsolver_json['data']['x_data']
    x_df.columns = ['T_hts_0', 'T_hts_1', 'T_hts_2', 'T_hts_3',
    "T_lts","T_fpsc","T_fpsc_s","T_vtsc","T_vtsc_s","T_pscf","T_pscr",
    "T_shx_psc_0", "T_shx_psc_1", "T_shx_psc_2", "T_shx_psc_3",
    "T_shx_ssc_0", "T_shx_ssc_1", "T_shx_ssc_2", "T_shx_ssc_3"]

    data_df = pd.merge(u_df, b_df, on='time_grid')
    data_df = data_df.merge(x_df, on='time_grid')
    return data_df

In [ ]:
def plot_openloop_traj() -> None:
    cfig, ax = plt.subplots(6, 2, sharex=True, figsize=(12,11))

    ax[0][0].plot(nlp_rel_data_df.index, nlp_rel_data_df.T_fpsc, color = "#d62728ff", alpha=0.5, linestyle=':')
    ax[0][0].plot(nlp_rel_data_df.index, nlp_rel_data_df.T_vtsc, color = "#d6272880", alpha=0.5, linestyle=':')
    ax[0][0].plot(miqp_data_df.index, miqp_data_df.T_fpsc, color = "#d62728ff", label = "$T_\mathsf{fpsc}$")
    ax[0][0].plot(miqp_data_df.index, miqp_data_df.T_vtsc, color = "#d6272880", label = "$T_\mathsf{vtsc}$")

    ax[0][1].plot(nlp_rel_data_df.index, nlp_rel_data_df.T_fpsc, color = "#d62728ff", label = "$T_\mathsf{fpsc}$", alpha=0.5, linestyle=':')
    ax[0][1].plot(nlp_rel_data_df.index, nlp_rel_data_df.T_vtsc, color = "#d6272880", label = "$T_\mathsf{vtsc}$", alpha=0.5, linestyle=':')
    ax[0][1].plot(nlp_bin_data_df.index, nlp_bin_data_df.T_fpsc, color = "#d62728ff", label = "$T_\mathsf{fpsc}$")
    ax[0][1].plot(nlp_bin_data_df.index, nlp_bin_data_df.T_vtsc, color = "#d6272880", label = "$T_\mathsf{vtsc}$")

    for ax_j in ax[0]:
        ax_j.set_ylim(0,105)
        ax_j.set_ylabel("Temp. (°C)")
        ax_j.legend(loc="upper right", framealpha=0.0)


    ax[1][0].plot(nlp_rel_data_df.index, nlp_rel_data_df.T_hts_0, color = "#d62728ff", alpha=0.5, linestyle=':')
    ax[1][0].plot(nlp_rel_data_df.index, nlp_rel_data_df.T_hts_3, color = "#d6272880", alpha=0.5, linestyle=':')
    ax[1][0].plot(miqp_data_df.index, miqp_data_df.T_hts_0, color="#d62728ff", label = "$T_\mathsf{ht,1}$")
    ax[1][0].plot(miqp_data_df.index, miqp_data_df.T_hts_3, color="#d6272880", label = "$T_\mathsf{ht,4}$")
    ax[1][0].set_ylim(35,100)
    ax[1][0].set_ylabel("Temp. (°C)")
    ax[1][0].legend(loc="upper left", ncol=4, framealpha=0.0, bbox_to_anchor=(0, 1.15, 1.1, 0.0))

    ax[1][1].plot(nlp_rel_data_df.index, nlp_rel_data_df.T_hts_0, color = "#d62728ff", alpha=0.5, linestyle=':')
    ax[1][1].plot(nlp_rel_data_df.index, nlp_rel_data_df.T_hts_3, color = "#d6272880", alpha=0.5, linestyle=':')
    ax[1][1].plot(nlp_bin_data_df.index, nlp_bin_data_df.T_hts_0, color="#d62728ff", label = "$T_\mathsf{ht,1}$")
    ax[1][1].plot(nlp_bin_data_df.index, nlp_bin_data_df.T_hts_3, color="#d6272880", label = "$T_\mathsf{ht,4}$")
    ax[1][1].set_ylim(35,100)
    ax[1][1].set_ylabel("Temp. (°C)")
    ax[1][1].legend(loc="upper left", ncol=4, framealpha=0.0, bbox_to_anchor=(0, 1.15, 1.1, 0.0))


    ax[2][0].plot(nlp_rel_data_df.index, nlp_rel_data_df.T_lts, color = "#1f77b4ff", alpha=0.5, linestyle=':')
    ax[2][0].plot(miqp_data_df.index, miqp_data_df.T_lts, color = "#1f77b4ff", label = "$T_\mathsf{lt,1}$")
    ax[2][0].set_ylim(5,25)
    ax[2][0].set_ylabel("Temp. (°C)")
    ax[2][0].legend(loc="upper left", ncol=2, framealpha=0.0)

    ax[2][1].plot(nlp_rel_data_df.index, nlp_rel_data_df.T_lts, color = "#1f77b4ff", alpha=0.5, linestyle=':')
    ax[2][1].plot(nlp_bin_data_df.index, nlp_bin_data_df.T_lts, color = "#1f77b4ff", label = "$T_\mathsf{lt,1}$")
    ax[2][1].set_ylim(5,25)
    ax[2][1].set_ylabel("Temp. (°C)")
    ax[2][1].legend(loc="upper left", ncol=2, framealpha=0.0)


    ax[3][0].step(nlp_rel_data_df.index, nlp_rel_data_df.b_ac, where="post", color = "#1f77b4ff", alpha=0.5, linestyle=':')
    ax[3][0].step(nlp_rel_data_df.index, nlp_rel_data_df.b_fc, where="post", color = "#ff7f0eff", alpha=0.5, linestyle=':')
    ax[3][0].step(nlp_rel_data_df.index, nlp_rel_data_df.b_hp, where="post", color = "C2", alpha=0.5, linestyle=':')
    ax[3][0].step(miqp_data_df.index, miqp_data_df.b_ac, where="post", color = "#1f77b4ff", label = "$b_\mathsf{acm}$")
    ax[3][0].step(miqp_data_df.index, miqp_data_df.b_fc, where="post", color = "#ff7f0eff", label = "$b_\mathsf{fc}$")
    ax[3][0].step(miqp_data_df.index, miqp_data_df.b_hp, where="post", color = "C2", label = "$b_\mathsf{hp}$")

    ax[3][1].step(nlp_rel_data_df.index, nlp_rel_data_df.b_ac, where="post", color = "#1f77b4ff", alpha=0.5, linestyle=':')
    ax[3][1].step(nlp_rel_data_df.index, nlp_rel_data_df.b_fc, where="post", color = "#ff7f0eff", alpha=0.5, linestyle=':')
    ax[3][1].step(nlp_rel_data_df.index, nlp_rel_data_df.b_hp, where="post", color = "C2", alpha=0.5, linestyle=':')
    ax[3][1].step(nlp_bin_data_df.index, nlp_bin_data_df.b_ac, where="post", color = "#1f77b4ff", label = "$b_\mathsf{acm}$")
    ax[3][1].step(nlp_bin_data_df.index, nlp_bin_data_df.b_fc, where="post", color = "#ff7f0eff", label = "$b_\mathsf{fc}$")
    ax[3][1].step(nlp_bin_data_df.index, nlp_bin_data_df.b_hp, where="post", color = "C2", label = "$b_\mathsf{hp}$")

    for ax_j in ax[3]:
        ax_j.set_ylim(0, 1.1)
        ax_j.set_ylabel("Status {0, 1}")
        ax_j.legend(loc="upper right", framealpha=0.0)


    ax[4][0].step(nlp_rel_data_df.index, nlp_rel_data_df.v_ppsc, where="post", color = "C0", alpha=0.5, linestyle=':')
    ax[4][0].step(nlp_rel_data_df.index, nlp_rel_data_df.v_pssc, where="post", color = "C1", alpha=0.5, linestyle=':')
    ax[4][0].step(miqp_data_df.index, miqp_data_df.v_ppsc, where="post", color = "C0", label="$v_\mathsf{ppsc}$")
    ax[4][0].step(miqp_data_df.index, miqp_data_df.v_pssc, where="post", color = "C1", label="$v_\mathsf{pssc}$")

    ax[4][1].step(nlp_rel_data_df.index, nlp_rel_data_df.v_ppsc, where="post", color = "C0", alpha=0.5, linestyle=':')
    ax[4][1].step(nlp_rel_data_df.index, nlp_rel_data_df.v_pssc, where="post", color = "C1", alpha=0.5, linestyle=':')
    ax[4][1].step(nlp_bin_data_df.index, nlp_bin_data_df.v_ppsc, where="post", color = "C0", label="$v_\mathsf{ppsc}$")
    ax[4][1].step(nlp_bin_data_df.index, nlp_bin_data_df.v_pssc, where="post", color = "C1", label="$v_\mathsf{pssc}$")


    for ax_j in ax[4]:
        ax_j.set_ylim(0, 1.1)
        ax_j.set_ylabel("Status [0, 1]")
        ax_j.legend(loc="upper right", framealpha=0.0)


    ax[5][0].step(nlp_rel_data_df.index, nlp_rel_data_df.p_mpsc, where="post", color = "C0", alpha=0.5, linestyle=':')
    mdot_o_hts_b_ba = nlp_rel_data_df.P_g / (nlp_rel_data_df.v_pssc*0.625)
    mdot_o_hts_b_ba[nlp_rel_data_df.v_pssc < 1e-2] = np.nan
    mdot_i_hts_b_ba = nlp_rel_data_df.mdot_o_hts_b / (nlp_rel_data_df.b_ac*(2496.0 / 3600.0))
    mdot_i_hts_b_ba[nlp_rel_data_df.b_ac < 1e-2] = np.nan
    ax[5][0].step(nlp_rel_data_df.index, mdot_o_hts_b_ba, where="post", color = "C1", alpha=0.5, linestyle=':')
    ax[5][0].step(nlp_rel_data_df.index, mdot_i_hts_b_ba, where="post", color = "C2", alpha=0.5, linestyle=':')

    ax[5][0].step(nlp_rel_data_df.index, nlp_rel_data_df.p_mpsc, where="post", color = "C0")
    mdot_o_hts_b_ba = miqp_data_df.P_g / (miqp_data_df.v_pssc*0.625)
    mdot_o_hts_b_ba[miqp_data_df.v_pssc < 1e-2] = np.nan
    mdot_i_hts_b_ba = miqp_data_df.mdot_o_hts_b / (miqp_data_df.b_ac*(2496.0 / 3600.0))
    mdot_i_hts_b_ba[miqp_data_df.b_ac < 1e-2] = np.nan
    ax[5][0].step(miqp_data_df.index, mdot_o_hts_b_ba, where="post", color = "C1", label=r"$\frac{\dot{m}_\mathsf{o,hts,b}}{\dot{m}_\mathsf{ssc}}$")
    ax[5][0].step(miqp_data_df.index, mdot_i_hts_b_ba, where="post", color = "C2", label=r"$\frac{\dot{m}_\mathrm{i,hts,b}}{\dot{m}_\mathrm{ac,ht}}$")

    ax[5][1].step(nlp_rel_data_df.index, nlp_rel_data_df.p_mpsc, where="post", color = "C0", alpha=0.5, linestyle=':')
    mdot_o_hts_b_ba = nlp_rel_data_df.P_g / (nlp_rel_data_df.v_pssc*0.625)
    mdot_o_hts_b_ba[nlp_rel_data_df.v_pssc < 1e-2] = np.nan
    mdot_i_hts_b_ba = nlp_rel_data_df.mdot_o_hts_b / (nlp_rel_data_df.b_ac*(2496.0 / 3600.0))
    mdot_i_hts_b_ba[nlp_rel_data_df.b_ac < 1e-2] = np.nan
    ax[5][1].step(nlp_rel_data_df.index, mdot_o_hts_b_ba, where="post", color = "C1", alpha=0.5, linestyle=':')
    ax[5][1].step(nlp_rel_data_df.index, mdot_i_hts_b_ba, where="post", color = "C2", alpha=0.5, linestyle=':')

    ax[5][1].step(nlp_bin_data_df.index, nlp_bin_data_df.p_mpsc, where="post", color = "C0", label="$p_\mathsf{mpsc}$")
    mdot_o_hts_b_bin = nlp_bin_data_df.P_g / (nlp_bin_data_df.v_pssc*0.625)
    mdot_o_hts_b_bin[nlp_bin_data_df.v_pssc < 1e-2] = np.nan
    mdot_i_hts_b_bin = nlp_bin_data_df.mdot_o_hts_b / (nlp_bin_data_df.b_ac*(2496.0 / 3600.0))
    mdot_i_hts_b_bin[nlp_bin_data_df.b_ac < 1e-2] = np.nan
    ax[5][1].step(nlp_bin_data_df.index, mdot_o_hts_b_bin, where="post", color = "C1", label=r"$\frac{\dot{m}_\mathsf{o,hts,b}}{\dot{m}_\mathsf{ssc}}$")
    ax[5][1].step(nlp_bin_data_df.index, mdot_i_hts_b_bin, where="post", color = "C2", label=r"$\frac{\dot{m}_\mathrm{i,hts,b}}{\dot{m}_\mathrm{ac,ht}}$")

    for ax_j in ax[5]:

        ax_j.set_ylim(0, 1.1)
        ax_j.set_ylabel("Status [0, 1]")

        lines = ax_j.get_lines()

        try:

            legend1 = ax_j.legend(lines, \
                                [r"$p_\mathsf{mpsc}$"], \
                                loc="lower left", framealpha=0.0)

            legend2 = ax_j.legend([lines[i] for i in [1, 2]], \
                                [r"$\frac{\dot{m}_\mathsf{o,hts,b}}{\dot{m}_\mathsf{ssc}}$", \
                                r"$\frac{\dot{m}_\mathrm{i,hts,b}}{\dot{m}_\mathrm{ac,ht}}$"], \
                                loc="lower right", framealpha=0.0)

            ax_j.add_artist(legend1)

        except IndexError:
            pass

    for ax_j in ax[-1]:
        ax_j.set_xlim(miqp_data_df.index[0], miqp_data_df.index[-1])
        ax_j.set_xlabel("Time (h)")
        loc = ax_j.xaxis.get_major_locator()
        loc.maxticks[mpl.dates.HOURLY] = 7

    for ax_k in ax:
        for ax_k_j in ax_k:
            ax_k_j.spines["top"].set_visible(False)
            ax_k_j.spines["right"].set_visible(False)

    ax[0][0].set_title("Step 1: Solution GN-MIQP")
    ax[0][1].set_title("Step 2: Solution MINLP binaries fixed")
    plt.gcf().suptitle(f"Voronoi iteration: {iter_nr}")

    plt.savefig(f"figures/voronoi_iteration_{iter_nr}.png")
    # plt.show();

In [ ]:
miqp_data_df = generate_data_df(nlpsolver_json=nlpsolver_miqp)
nlp_bin_data_df = generate_data_df(nlpsolver_json=nlpsolver_nlp_bin)
nlp_rel_data_df = generate_data_df(nlpsolver_json=nlpsolver_rel)

In [ ]:
c_df = pd.DataFrame(columns=['time_grid']+[f'c_{i}' for i in range(len(nlpsolver_miqp['data']['c_data'][0]))])
c_df['time_grid'] = nlpsolver_miqp['time_points']
c_df.set_index('time_grid', inplace=True)
c_df.index = pd.to_datetime(c_df.index).tz_convert('CET')
c_df[c_df.columns] = nlpsolver_miqp['data']['c_data']
c_df.columns = ["T_amb","I_fpsc","I_vtsc","Qdot_c","P_pv_kWp","p_g"]
c_df.head()

### Ambient

In [ ]:
fig, ax = plt.subplots(3, 1, sharex=True, figsize=(10,7))

ax[0].plot(c_df.index, c_df.I_fpsc/1e3, color = "#d62728ff", label = "$I_\mathsf{fpsc}$")
ax[0].plot(c_df.index, c_df.I_vtsc/1e3, color = "#8c0e6ca0", label = "$I_\mathsf{vtsc}$")
ax0a = ax[0].twinx()
ax0a.plot(c_df.index, c_df.P_pv_kWp/1e3, color = "C2", label = "$P_\mathsf{pv,kWp}$")

ax[0].set_ylim(0,1)
ax[0].set_ylabel("Irrad. (kW/m²)")
ax[0].legend(loc="upper left", framealpha=0.0)

ax0a.set_ylim(0,1)
ax0a.set_ylabel("Power kW")
ax0a.legend(loc="upper right", framealpha=0.0)

ax[1].plot(c_df.index, c_df.T_amb, color = "C0", label = "$T_\mathsf{amb}$")
#ax[0].set_ylim(0,950)
ax[1].set_ylabel("Temp. (°C)", labelpad=12)
ax[1].set_ylim(10, 30)
ax[1].legend(loc="lower center", framealpha=0.0,  bbox_to_anchor=(0.35, 0.05))

ax1a = ax[1].twinx()
ax1a.plot(c_df.index, c_df.Qdot_c/1e3, color = "C1", label = "$\dot{Q}_\mathsf{lc}$")
ax1a.set_ylabel("Load (kW)")
ax1a.set_ylim(0, 9)
ax1a.legend(loc="upper right", framealpha=0.0)

ax[2].plot(c_df.index, c_df.p_g, color = "C0", label = "$p_\mathsf{g}$")
ax[2].set_ylim(0,20)
ax[2].set_ylabel("Price (ct/kWh)")
ax[2].legend(loc="upper right", framealpha=0.0)

ax[-1].set_xlim(c_df.index[0], c_df.index[-1])
ax[-1].set_xlabel("Time (h)")

for ax_k in ax:
    ax_k.spines["top"].set_visible(False)
    ax_k.spines["right"].set_visible(False)

ax1a.spines["top"].set_visible(False)

plt.show();

### Open-loop trajectories

In [ ]:
plot_openloop_traj()

## Generates all the figures

In [ ]:
%%capture
for iter_nr in range(71):
    with open(RESULTS_DIR + MIQP_NAME(iter_nr=iter_nr), "rb") as f:
        nlpsolver_miqp = json.load(f)
    with open(RESULTS_DIR + NLP_NAME(iter_nr=iter_nr), "rb") as f:
            nlpsolver_nlp_bin = json.load(f)

    miqp_data_df = generate_data_df(nlpsolver_json=nlpsolver_miqp)
    nlp_bin_data_df = generate_data_df(nlpsolver_json=nlpsolver_nlp_bin)
    nlp_rel_data_df = generate_data_df(nlpsolver_json=nlpsolver_rel)

    plot_openloop_traj()

---
## Backup cells

In [ ]:
raise

In [ ]:
u_df = pd.DataFrame(columns=['time_grid']+[f'u_{i}' for i in range(len(nlpsolver_miqp['data']['u_data'][0]))])
u_df['time_grid'] = nlpsolver_miqp['time_points']
u_df.set_index('time_grid', inplace=True)
u_df.index = pd.to_datetime(u_df.index).tz_convert('CET')
tmp = copy.copy(nlpsolver_miqp['data']['u_data'])
tmp.append((np.nan * np.zeros(len(nlpsolver_miqp['data']['u_data'][0]))).tolist())
u_df[u_df.columns] = tmp
u_df.columns = ["v_ppsc", "p_mpsc", "v_pssc", "P_g", "mdot_o_hts_b", "mdot_i_hts_b"]
u_df.head()

In [ ]:
b_df = pd.DataFrame(columns=['time_grid']+[f'b_{i}' for i in range(len(nlpsolver_miqp['data']['b_data'][0]))])
b_df['time_grid'] = nlpsolver_miqp['time_points']
b_df.set_index('time_grid', inplace=True)
b_df.index = pd.to_datetime(b_df.index).tz_convert('CET')
tmp = copy.copy(nlpsolver_miqp['data']['b_data'])
tmp.append((np.nan * np.zeros(len(nlpsolver_miqp['data']['b_data'][0]))).tolist())
b_df[b_df.columns] = tmp
b_df.columns = ["b_ac", "b_fc", "b_hp"]
b_df.head()

In [ ]:
x_df = pd.DataFrame(columns=['time_grid']+[f'x_{i}' for i in range(len(nlpsolver_miqp['data']['x_data'][0]))])
x_df['time_grid'] = nlpsolver_miqp['time_points']
x_df.set_index('time_grid', inplace=True)
x_df.index = pd.to_datetime(x_df.index).tz_convert('CET')
x_df[x_df.columns] = nlpsolver_miqp['data']['x_data']
x_df.columns = ['T_hts_0', 'T_hts_1', 'T_hts_2', 'T_hts_3',
"T_lts","T_fpsc","T_fpsc_s","T_vtsc","T_vtsc_s","T_pscf","T_pscr",
"T_shx_psc_0", "T_shx_psc_1", "T_shx_psc_2", "T_shx_psc_3",
"T_shx_ssc_0", "T_shx_ssc_1", "T_shx_ssc_2", "T_shx_ssc_3"]
x_df.head()